In [2]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# set seed for reproducibility
np.random.seed(0)

In [4]:
#we will use PakistanSuicideAttacks Ver 11(30-November-2017.csv)

with open ("PakistanSuicideAttacks Ver 11 (30-November-2017).csv","rb") as rawdata:
    result = chardet.detect(rawdata.read(100000))
    
print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [5]:
#so the encoding is Windows-1252 ! , lets read the data :
suicide_attacks = pd.read_csv("PakistanSuicideAttacks Ver 11 (30-November-2017).csv",encoding="Windows-1252")

#read the first few rows:
suicide_attacks.head()

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,...,None,14.0,15.0,NaN,60,2.0,NaN,NaN,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,...,None,NaN,3.0,NaN,3,1.0,NaN,NaN,23.770,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,13.0,15.0,20.0,40,1.0,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,Friday-June 14-2002,3 Raby` al-THaany 1423 A.H,Working Day,NaN,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,NaN,12.0,NaN,51,1.0,NaN,NaN,31.430,88.574
4,5,Friday-July 4-2003,4 Jumaada al-awal 1424 A.H,Working Day,NaN,NaN,Quetta,30.2095,67.0182,Baluchistan,...,Shiite,44.0,47.0,NaN,65,1.0,NaN,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616


In [6]:
#we will clean up the City column 
#get all the unique values im the 'City' column
cities = suicide_attacks['City'].unique()


#sort them alphabetically and check if there is inconsistent data entry :
cities.sort()
cities

#we can see duplicated cities with and without upercases and lowercases

array(['ATTOCK', 'Attock ', 'Bajaur Agency', 'Bannu', 'Bhakkar ', 'Buner',
       'Chakwal ', 'Chaman', 'Charsadda', 'Charsadda ', 'D. I Khan',
       'D.G Khan', 'D.G Khan ', 'D.I Khan', 'D.I Khan ', 'Dara Adam Khel',
       'Dara Adam khel', 'Fateh Jang', 'Ghallanai, Mohmand Agency ',
       'Gujrat', 'Hangu', 'Haripur', 'Hayatabad', 'Islamabad',
       'Islamabad ', 'Jacobabad', 'KURRAM AGENCY', 'Karachi', 'Karachi ',
       'Karak', 'Khanewal', 'Khuzdar', 'Khyber Agency', 'Khyber Agency ',
       'Kohat', 'Kohat ', 'Kuram Agency ', 'Lahore', 'Lahore ',
       'Lakki Marwat', 'Lakki marwat', 'Lasbela', 'Lower Dir', 'MULTAN',
       'Malakand ', 'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mohmand Agency ', 'Mohmand agency', 'Mosal Kor, Mohmand Agency',
       'Multan', 'Muzaffarabad', 'North Waziristan', 'North waziristan',
       'Nowshehra', 'Orakzai Agency', 'Peshawar', 'Peshawar ', 'Pishin',
       'Poonch', 'Quetta', 'Quetta ', 'Rawalpindi', 'Sargodha',
       'Sehwan town',

In [7]:
#converting all data to lowercases and remove the spaces :
suicide_attacks['City'] = suicide_attacks['City'].str.lower()

#remove the trailing white spaces :
suicide_attacks['City'] = suicide_attacks['City'].str.strip()

#check the first few rows:
suicide_attacks['City'].head()

0    islamabad
1      karachi
2      karachi
3      karachi
4       quetta
Name: City, dtype: object

In [8]:
# Then convert the column to lowercase and remove any trailing white spaces

Province = suicide_attacks['Province'].unique()
Province.sort()
Province

#as we can see , there is inconsistencies "FATA" , "Fata"(same)
#and "Balochistan" , "Baluchistan" (maybe same with spelling mistakes!)
#no trailing white spaces 

#convert the Province column to lower case :
suicide_attacks['Province'] = suicide_attacks['Province'].str.lower()

#remove the trailing white spaces (if exist)
suicide_attacks['Province'] = suicide_attacks['Province'].str.strip()

suicide_attacks['Province'].unique()

array(['capital', 'sindh', 'baluchistan', 'punjab', 'fata', 'kpk', 'ajk',
       'balochistan'], dtype=object)

In [9]:
#Use fuzzy matching to correct inconsistent data entry¶
#take a look on city column after what we did before :

cities = suicide_attacks['City'].unique()

#sort it again :
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd. i khan', 'd.g khan', 'd.i khan',
       'dara adam khel', 'fateh jang', 'ghallanai, mohmand agency',
       'gujrat', 'hangu', 'haripur', 'hayatabad', 'islamabad', 'jacobabad',
       'karachi', 'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'], dtype=object)

In [10]:
#'d. i khan' and 'd.i khan' in City column is same !
#but we dont know if 'd.g khan' is same city as previous or new one 
#by checking online , its new :) 
#Here, we're going to get the ten strings from our list of cities that have the closest distance to "d.i khan".

#get the top 10 closest matchs to "d.i khan"
matches = fuzzywuzzy.process.extract("d.i khan", cities,limit=10
                        ,scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('d. i khan', 100),
 ('d.i khan', 100),
 ('d.g khan', 88),
 ('khanewal', 50),
 ('sudhanoti', 47),
 ('hangu', 46),
 ('kohat', 46),
 ('dara adam khel', 45),
 ('chaman', 43),
 ('mardan', 43)]

In [11]:
# write a function :

# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string

def replace_matches_in_column(df, column, string_to_match,min_ratio=90):
    #get the list of unique string 
    strings = df[column].unique()
    matches = fuzzywuzzy.process.extract(string_to_match,strings,limit=10,
                                         scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    #get only the matches for ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    
    #get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    
    #replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    #test if the function is working :
    print("done!")

In [12]:
## use the function we just wrote to replace close matches to "d.i khan" with "d.i khan"
replace_matches_in_column(df=suicide_attacks,column = 'City', string_to_match='d.i khan')

done!


In [13]:
#checking again the unique values in City column and check the ratio :

#get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

#sort them alphabetically :
cities.sort()
cities

#from the results: we have only 1 'd.i khan' 

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'], dtype=object)

In [14]:


# Your turn! It looks like 'kuram agency' and 'kurram agency' should
# be the same city. Correct the dataframe so that they are.

#by checking , the city name should be 'kurram agency', and for that
#we should replace the closest match (which is in our case 'kuram agency')
#with input match 'kurram agency'

#we can use the same function to replace but first lets get the 
#most 10 closest match to 'kurram agency'
matches2 = fuzzywuzzy.process.extract("kurram agency", cities,limit=10
                        ,scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches2



[('kurram agency', 100),
 ('kuram agency', 96),
 ('bajaur agency', 69),
 ('khyber agency', 69),
 ('orakzai agency', 67),
 ('mohmand agency', 59),
 ('mosal kor, mohmand agency', 59),
 ('ghallanai, mohmand agency', 49),
 ('gujrat', 42),
 ('d.g khan', 38)]

In [15]:
#we cot 96% ratio, now lets call the function :

replace_matches_in_column(df=suicide_attacks, column='City', string_to_match='kurram agency')

done!


In [16]:
#now lets check again if the changes is reflected :
cities = suicide_attacks['City'].unique()

cities.sort()
cities

#it worked! , congrats !!!

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kurram agency', 'lahore', 'lakki marwat', 'lasbela', 'lower dir',
       'malakand', 'mansehra', 'mardan', 'mohmand agency',
       'mosal kor, mohmand agency', 'multan', 'muzaffarabad',
       'north waziristan', 'nowshehra', 'orakzai agency', 'peshawar',
       'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'], dtype=object)

In [ ]:
#********************* Finish Project *********************#